In [1]:
!pip install torch torchvision scikit-learn matplotlib pandas


In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt


In [3]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
sample_sub = pd.read_csv("data/sample_submission.csv")

print(train_df.head())


        id_code  diagnosis
0  000c1434d8d7          2
1  001639a390f0          4
2  0024cdab0c1e          1
3  002c21358ce6          0
4  005b95c28852          0


In [4]:
train_df['filepath'] = train_df['id_code'].apply(lambda x: f"data/train_images/{x}.png")
test_df['filepath'] = test_df['id_code'].apply(lambda x: f"data/test_images/{x}.png")


In [5]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std= [0.229, 0.224, 0.225])
])


In [6]:
class RetinopathyDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image = Image.open(row['filepath']).convert('RGB')
        label = torch.tensor(row['diagnosis'], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label


In [7]:
train_size = int(0.8 * len(train_df))
val_size   = len(train_df) - train_size

train_subset, val_subset = random_split(train_df, [train_size, val_size],
                                        generator=torch.Generator().manual_seed(42))

train_ds = RetinopathyDataset(train_subset.dataset.loc[train_subset.indices], transform=data_transforms)
val_ds   = RetinopathyDataset(val_subset.dataset.loc[val_subset.indices], transform=data_transforms)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0)


In [8]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last layers for fine-tuning
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:
        param.requires_grad = True

# Replace final FC layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [9]:
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(train_df['diagnosis']),
                                     y=train_df['diagnosis'])
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [10]:
from torch.optim.lr_scheduler import StepLR
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)


In [11]:
# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze only the last layers for initial training
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True


In [12]:
num_epochs = 5  # initial training with top layers

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()  # adjust learning rate
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")


Epoch 1, Loss: 237.4454
Epoch 2, Loss: 185.0410
Epoch 3, Loss: 164.0999
Epoch 4, Loss: 152.4620
Epoch 5, Loss: 147.8090


In [13]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")


Validation Accuracy: 72.99%
